In [1]:
import sys
sys.path.insert(0,"../")
import numpy as np
import PyGT

## Load in matrix and vectors selecting $\mathcal{A,B}$ regions using the KTN format

KTN (Kinetic Transition Network) format requires two files:
- `min.data` columns :  $E,\,2S/{\rm k_B},\,D,\,I_x,\,I_y,\,I_z$
- `ts.data`  columns    :  $E,\,2S/{\rm k_B},\,D,\,f,\,i,\,I_x,\,I_y,\,I_z$

where 
- $E$ = Energy
- $S$ = Entropy
- $D$ = Degeneracy
- $I_x$ = $x$-moment of intertia
- $f,i$ = final,initial state indicies

Some technical details:
- `load_ktn()` function prunes isolated nodes, changing the node indices 
- `load_ktn_AB()` function looks for files `min.[A,B]` which use the same indicies as the data file
- `retained` is a vector that maps from the unpruned to pruned indexing convention
- `A_vec, B_vec` can clearly be determined without using `load_ktn_AB()`

In [2]:
data_path = "../KTN_data/LJ38/4k/"
temp = 1.
beta = 1./temp

B, K, tau, N, u, s, Emin, retained = PyGT.io.load_ktn(path=data_path,beta=beta,screen=True)

F = u - s/beta # free energy

A_vec, B_vec = PyGT.io.load_ktn_AB(data_path,retained)
I_vec = ~(A_vec+B_vec)
print(f'States in A,I,B: {A_vec.sum(),I_vec.sum(),B_vec.sum()}')


	Connected Clusters: 1620, of which 95% have <= 2 states
	Retaining largest cluster with 900 nodes

States in A,I,B: (5, 767, 128)


# Remove a set of nodes in $\mathcal{I}$ using graph transformation

- We remove all nodes in $\mathcal{I}$ above the 10th percentile in free energy


In [3]:

rm_vec = np.zeros(N,bool)

rm_vec[I_vec] = F[I_vec]>np.percentile(F[I_vec],10.0)

gt_B, gt_tau, gt_K = PyGT.GT(rm_vec,B,tau,block=10,rates=True,screen=True)



GT done in 0.81 seconds with 0 floating point corrections
